In [15]:
#import
import pandas as pd
from gensim.models.fasttext import FastText
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
#read data
df = pd.read_csv('finnum/train.csv')
x = df.index
dfval = pd.read_csv('finnum/dev.csv')

In [5]:
df['cat_num'] = df['category'].astype('category').cat.codes
dfval['cat_num'] = dfval['category'].astype('category').cat.codes

In [6]:
#load embeddings
model = FastText.load('Gensim FastText Vectors/fastText1')

In [7]:
#lowcase all text
df['lower'] = [x.lower() for x in df.tweet]
dfval['lower'] = [x.lower() for x in dfval.tweet]

In [8]:
#function to put flags for numbers in embedding
import re
def flagNum(x):
    text = x.lower
    outNum = str(x.target_num)
    text_out = re.sub(r'(?<=\D)'+outNum+'(?=\D)', ' <num> ', text)
    #text_out = text.replace('\D('+outNum+')\D', ' <num> ')
    return text_out

In [9]:
#Add flags to columns
df['mod'] = df.apply(lambda x: flagNum(x), axis = 1)
dfval['mod'] = dfval.apply(lambda x: flagNum(x), axis = 1)

In [10]:
#function to clean punctuation and small words
import string
def textPuncandNum(text):
    table = str.maketrans('', '', string.punctuation)
    text = re.sub(r'[0-9]+', '', text)
    text = text.split()
    text = [word for word in text if len(word.translate(table))>2]
    return ' '.join(text)


In [11]:
#function to compute average of word embeddings

def computeavg(tweet):
    splits = tweet.split()
    ysplit = model.wv[splits]
    m = np.mean(ysplit, axis = 0)
    return m


In [12]:
#apply filter and average

df['filt'] = df['mod'].apply(textPuncandNum)
df['avg'] = df['filt'].apply(computeavg)
dfval['filt'] = dfval['mod'].apply(textPuncandNum)
dfval['avg'] = dfval['filt'].apply(computeavg)
dfval.head()

,id,idx,tweet,category,subcategory,target_num,cat_num,lower,mod,filt,avg
0,103568095,1302,$THCX.V One of the better setups.{C&amp;H} set...,Monetary,buy price,3.01,1,$thcx.v one of the better setups.{c&amp;h} set...,$thcx.v one of the better setups.{c&amp;h} set...,$thcx.v one the better setups.{c&amp;h} setup ...,"[0.20721708, 0.09418728, -0.41839322, -0.21092..."
1,103568095,1302,$THCX.V One of the better setups.{C&amp;H} set...,Monetary,stop loss,2.45,1,$thcx.v one of the better setups.{c&amp;h} set...,$thcx.v one of the better setups.{c&amp;h} set...,$thcx.v one the better setups.{c&amp;h} setup ...,"[0.20539609, 0.1008702, -0.42466328, -0.203908..."
2,103745329,2416,"$TQQQ welp, got stopped at 130.7. $soxl $smh f...",Monetary,stop loss,130.7,1,"$tqqq welp, got stopped at 130.7. $soxl $smh f...","$tqqq welp, got stopped at <num> . $soxl $smh...","$tqqq welp, got stopped <num> $soxl $smh finis...","[0.23352587, -1.0280132, -1.401173, -0.3327945..."
3,103745329,2416,"$TQQQ welp, got stopped at 130.7. $soxl $smh f...",Indicator,Indicator,20,0,"$tqqq welp, got stopped at 130.7. $soxl $smh f...","$tqqq welp, got stopped at 130.7. $soxl $smh f...","$tqqq welp, got stopped $soxl $smh finishing w...","[0.23352587, -1.0280132, -1.401173, -0.3327944..."
4,103745329,2416,"$TQQQ welp, got stopped at 130.7. $soxl $smh f...",Temporal,date,3,6,"$tqqq welp, got stopped at 130.7. $soxl $smh f...","$tqqq welp, got stopped at 130.7. $soxl $smh f...","$tqqq welp, got stopped $soxl $smh finishing w...","[0.23352587, -1.0280132, -1.401173, -0.3327944..."


In [13]:
#make each dimension a feature
df3 = pd.DataFrame(df['avg'].values.tolist())
df3val = pd.DataFrame(dfval['avg'].values.tolist())
df3val.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.207217,0.094187,-0.418393,-0.210922,-1.563378,-0.897995,2.240289,-0.265143,0.396256,-0.321941,...,0.145929,1.254953,0.947551,0.702230,-0.868596,-1.730349,-1.458312,1.008027,0.414126,-0.127519
1,0.205396,0.100870,-0.424663,-0.203908,-1.568591,-0.888313,2.243914,-0.271325,0.407276,-0.328399,...,0.154528,1.254947,0.957327,0.715269,-0.867153,-1.732876,-1.469105,1.015223,0.416374,-0.128606
2,0.233526,-1.028013,-1.401173,-0.332795,-0.978836,-1.445791,1.118072,-0.077661,-0.611202,-0.280414,...,0.198882,0.887056,0.506800,0.513897,-0.369124,-1.586927,-0.038395,-0.151955,0.024986,0.464140
3,0.233526,-1.028013,-1.401173,-0.332794,-0.978836,-1.445791,1.118072,-0.077661,-0.611202,-0.280414,...,0.198882,0.887056,0.506800,0.513897,-0.369124,-1.586928,-0.038395,-0.151955,0.024986,0.464140
4,0.233526,-1.028013,-1.401173,-0.332794,-0.978836,-1.445791,1.118072,-0.077661,-0.611202,-0.280414,...,0.198882,0.887056,0.506800,0.513897,-0.369124,-1.586927,-0.038395,-0.151955,0.024986,0.464140


In [14]:
#gather y axis
yfull = df['category']
yfullval = dfval['category']
yfull.head()

0    Product Number
1          Monetary
2    Product Number
3    Product Number
4        Percentage
Name: category, dtype: object

In [16]:
#declare random forest model
clf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)

In [17]:
#fit random forest on train set
clf.fit(df3, yfull)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=2, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [18]:
#make preds on test set
preds = clf.predict(df3val)

In [19]:
#to DF
predsdf = pd.DataFrame(preds)
predsdf.head()

,0
0,Monetary
1,Monetary
2,Monetary
3,Monetary
4,Monetary


In [20]:
#compute accuracy
accuracy_score(yfullval, preds)

0.4206989247311828

In [22]:
#look at value counts
predsdf[0].value_counts()

Monetary    537
Temporal    207
Name: 0, dtype: int64